In [404]:
import duckdb

con = duckdb.connect(database=":memory:")

con.sql("""
CREATE TABLE beslut AS
    FROM 'data/resultat-ansokningsomgang-2020-2024-beslut.csv';
CREATE TABLE diarie AS
    FROM 'data/resultat-ansokningsomgang-2020-2024-diarie_kommun.csv';
""")

In [405]:
con.sql("""describe beslut;""").show()

┌─────────────────────────────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┐
│             column_name             │ column_type │  null   │   key   │ default │  extra  │
│               varchar               │   varchar   │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┤
│ Utbildningsområde                   │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Utbildningsnamn                     │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Län                                 │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Kommun                              │ VARCHAR     │ YES     │ NULL    │ NULL    │ NULL    │
│ Antal kommuner                      │ BIGINT      │ YES     │ NULL    │ NULL    │ NULL    │
│ Flera kommuner                      │ BOOLEAN     │ YES     │ NULL    │ NULL    │ NULL    │
│ YH-poäng                            │ BIGINT      │ YES   

In [406]:
con.sql("""select * from beslut;""").describe()

┌─────────┬───────────────────┬────────────────────────────────────────────────────────────────────┬──────────────┬──────────────┬────────────────────┬────────────────┬───────────────────┬────────────┬────────────────────┬─────────────────────────────────┬───────────────┬───────────────────────────┬───────────────────────────────┬──────────────────────┬──────────────────────────┬─────────────────────────────────────┬────────────────────┬───────────────┬─────────┐
│  aggr   │ Utbildningsområde │                          Utbildningsnamn                           │     Län      │    Kommun    │   Antal kommuner   │ Flera kommuner │     YH-poäng      │ Studieform │    Studietakt %    │      Utbildningsanordnare       │ Huvudmannatyp │ Sökta utbildningsomgångar │ Beviljade utbildningsomgångar │ Sökta platser totalt │ Beviljade platser totalt │ Sökta platser per utbildningsomgång │  Ansökningsomgång  │ Diarienummer  │ Beslut  │
│ varchar │      varchar      │                              var

In [407]:
con.sql("""
        select
            "Utbildningsanordnare",
            count ("Diarienummer") as "diarienummer_count",
            sum(case when "Beslut" = true then 1 else 0 end) as beviljad_count,
            round(100.0 * sum(case when "Beslut" = true then 1 else 0 end) / count(*), 2) as beviljad_pct
        from beslut
        group by "Utbildningsanordnare"
        order by "diarienummer_count" desc, "Utbildningsanordnare";
        """).show()

┌───────────────────────────────────────────────┬────────────────────┬────────────────┬──────────────┐
│             Utbildningsanordnare              │ diarienummer_count │ beviljad_count │ beviljad_pct │
│                    varchar                    │       int64        │     int128     │    double    │
├───────────────────────────────────────────────┼────────────────────┼────────────────┼──────────────┤
│ Företagsekonomiska Institutet 1888 AB         │                297 │             42 │        14.14 │
│ TUC Sweden AB - Yrkeshögskola                 │                280 │            115 │        41.07 │
│ Lernia Utbildning AB                          │                238 │             75 │        31.51 │
│ JENSEN Education School AB                    │                230 │             27 │        11.74 │
│ KYH AB                                        │                225 │             87 │        38.67 │
│ YrkesAkademin YH AB                           │                222 │   

In [408]:
con.sql("""
        select
            "Utbildningsanordnare",
            "Ansökningsomgång",
            count ("Diarienummer") as "diarienummer_count",
            sum(case when "Beslut" = true then 1 else 0 end) as beviljad_count,
            round(100.0 * sum(case when "Beslut" = true then 1 else 0 end) / count(*), 2) as beviljad_pct
        from beslut
        group by "Utbildningsanordnare", "Ansökningsomgång"
        order by "beviljad_count" desc, "Utbildningsanordnare", "Ansökningsomgång";
        """).show()

┌─────────────────────────────────────────────────────────┬──────────────────┬────────────────────┬────────────────┬──────────────┐
│                  Utbildningsanordnare                   │ Ansökningsomgång │ diarienummer_count │ beviljad_count │ beviljad_pct │
│                         varchar                         │      int64       │       int64        │     int128     │    double    │
├─────────────────────────────────────────────────────────┼──────────────────┼────────────────────┼────────────────┼──────────────┤
│ YrkesAkademin YH AB                                     │             2022 │                 68 │             40 │        58.82 │
│ KYH AB                                                  │             2022 │                 74 │             32 │        43.24 │
│ YrkesAkademin YH AB                                     │             2021 │                 65 │             31 │        47.69 │
│ TUC Sweden AB - Yrkeshögskola                           │             2021

In [409]:
con.sql("""
        select
            "Diarienummer",
            count(*) as "Antal kommuner",
            sum(case when "Beslut" = true then 1 else 0 end) as "Antal beviljade",
            round(100.0 * sum(case when "Beslut" = true then 1 else 0 end) / count(*), 2) as "Beviljade %"
        from diarie
        where "Diarienummer" = 'MYH 2024/4220'
        group by "Diarienummer"
        """)

┌───────────────┬────────────────┬─────────────────┬─────────────┐
│ Diarienummer  │ Antal kommuner │ Antal beviljade │ Beviljade % │
│    varchar    │     int64      │     int128      │   double    │
├───────────────┼────────────────┼─────────────────┼─────────────┤
│ MYH 2024/4220 │              3 │               3 │       100.0 │
└───────────────┴────────────────┴─────────────────┴─────────────┘

In [410]:
con.sql("""
SELECT
    "Diarienummer",
    COUNT(*) AS "Antal kommuner",
    SUM(CASE WHEN "Beslut" = TRUE THEN 1 ELSE 0 END) AS "Antal beviljade",
    SUM(CASE WHEN "Beslut" = FALSE THEN 1 ELSE 0 END) AS "Antal avslag",
    ROUND(100.0 * SUM(CASE WHEN "Beslut" = TRUE THEN 1 ELSE 0 END) / COUNT(*), 2) AS "Beviljade %"
FROM diarie
GROUP BY "Diarienummer"
HAVING 
    SUM(CASE WHEN "Beslut" = TRUE THEN 1 ELSE 0 END) > 0
    AND
    SUM(CASE WHEN "Beslut" = FALSE THEN 1 ELSE 0 END) > 0
ORDER BY "Antal kommuner" DESC;
        """).show()


┌──────────────┬────────────────┬─────────────────┬──────────────┬─────────────┐
│ Diarienummer │ Antal kommuner │ Antal beviljade │ Antal avslag │ Beviljade % │
│   varchar    │     int64      │     int128      │    int128    │   double    │
├──────────────┴────────────────┴─────────────────┴──────────────┴─────────────┤
│                                    0 rows                                    │
└──────────────────────────────────────────────────────────────────────────────┘



In [411]:
con.sql("""
SELECT
    "Diarienummer",
    COUNT(*) AS "Antal kommuner",
    MIN("Beslut") AS "Min_beslut",
    MAX("Beslut") AS "Max_beslut",
    CASE 
        WHEN MIN("Beslut") = MAX("Beslut") THEN 'Consistent'
        ELSE 'Mixed'
    END AS "Decision_pattern"
FROM diarie
GROUP BY "Diarienummer"
HAVING "Decision_pattern" = 'Mixed';
        """).show()


┌──────────────┬────────────────┬────────────┬────────────┬──────────────────┐
│ Diarienummer │ Antal kommuner │ Min_beslut │ Max_beslut │ Decision_pattern │
│   varchar    │     int64      │  boolean   │  boolean   │     varchar      │
├──────────────┴────────────────┴────────────┴────────────┴──────────────────┤
│                                   0 rows                                   │
└────────────────────────────────────────────────────────────────────────────┘



In [412]:
# con.close()
